# Mongo DB

In [1]:
!pip install pymongo
from pymongo import collection

                                              0.0/382.5 kB ? eta -:--:--
     ------------------------------------- 382.5/382.5 kB 24.8 MB/s eta 0:00:00
                                              0.0/283.7 kB ? eta -:--:--
     ------------------------------------- 283.7/283.7 kB 17.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Query with conditions
We can retrieve documents that match certain conditions:

In [ ]:
# Find documents where 'age' is greater than 30
query = {"age": {"$gt": 30}}
docs = collection.find(query)
for doc in docs:
    print(doc)

## Sorting results
We can also sort the results of our queries:

In [ ]:
# Sort documents by 'age' in descending order
docs = collection.find().sort("age", -1)
for doc in docs:
    print(doc)

## Limiting results
If we only want a specific number of results:

In [ ]:
# Get the first 5 documents
docs = collection.find().limit(5)
for doc in docs:
    print(doc)

## Counting documents
We can count the number of documents that match a certain condition:

In [ ]:
# Count the number of documents where 'age' is greater than 30
count = collection.count_documents({"age": {"$gt": 30}})
print(count)

## Indexing
To optimize queries, we can create indexes:

In [ ]:
# Create an index on the 'age' field
collection.create_index("age")

## Aggregation
MongoDB also supports complex aggregations:

In [ ]:
# Group documents by 'city' and get the average age in each city
pipeline = [
    {"$group": {"_id": "$city", "average_age": {"$avg": "$age"}}}
]
results = collection.aggregate(pipeline)
for result in results:
    print(result)

## Error Handling
We can add some error handling to our database interactions to make our code more robust:

In [ ]:
from pymongo.errors import ConnectionFailure

try:
    # Try to establish a connection
    client = MongoClient('mongodb://localhost:27017/')
    client.admin.command('ismaster')
except ConnectionFailure:
    print("Server not available")

Close the connection when you're done:

In [ ]:
# Close the connection
client.close()

MongoEngine is a Document-Object Mapper (think ORM, but for document databases) for working with MongoDB from Python. It uses a simple declarative API, similar to the Django ORM.

This is a basic overview of how you can use MongoEngine to interact with MongoDB in a Pythonic and ORM-like way. Remember that MongoDB is a document database and lacks some features of relational databases, but MongoEngine does a good job abstracting the MongoDB operations in an easy-to-use API.

In [ ]:
!pip install mongoengine

## Connecting to MongoDB
Next, you'll need to connect to your MongoDB database:

In [ ]:
from mongoengine import connect

connect('mydatabase', host='localhost', port=27017)

## Defining a Document Schema
Now, you can define a schema for your documents. In MongoEngine, these are defined as classes that subclass mongoengine.Document:

In [ ]:
from mongoengine import Document, StringField, IntField

class User(Document):
    name = StringField(required=True, max_length=200)
    age = IntField(required=True)


## Inserting Documents
To insert a document, you create an instance of your Document subclass and call .save() on it:

In [ ]:
user = User(name='John', age=30)
user.save()

## Querying for Documents
Querying is also straightforward:

In [ ]:
# Find all users
users = User.objects()
for user in users:
    print(user.name, user.age)

# Find all users older than 20
users = User.objects(age__gt=20)
for user in users:
    print(user.name, user.age)


## Updating Documents
Updating a document can be achieved as follows:

In [ ]:
# Updating a single document
user = User.objects(name='John').first()
user.age = 31
user.save()

# Updating multiple documents
User.objects(age__lt=30).update(set__age=30)


## Deleting Documents
And finally, to delete documents:

In [ ]:
# Deleting a single document
user = User.objects(name='John').first()
user.delete()

# Deleting multiple documents
User.objects(age__lt=30).delete()


## Blog Example

In [ ]:
from mongoengine import Document, EmbeddedDocument, StringField, ReferenceField, ListField, DateTimeField, IntField
from datetime import datetime

class User(Document):
    name = StringField(required=True, max_length=200)
    age = IntField(required=True)

class Comment(EmbeddedDocument):
    content = StringField()
    author = ReferenceField(User)
    posted_at = DateTimeField(default=datetime.utcnow)

class Post(Document):
    title = StringField(max_length=120, required=True)
    author = ReferenceField(User)
    posted_at = DateTimeField(default=datetime.utcnow)
    comments = ListField(EmbeddedDocumentField(Comment))


In this example, a User is a document that has a name and age. A Comment is an embedded document — it doesn't exist on its own, but is always part of a Post. The Comment has a reference to the User who authored it, stored as a MongoDB ObjectId. A Post has a list of Comment embedded documents.

Now, let's create a user, a post, and add a comment to the post:

In [ ]:
# Create a new user
user = User(name='John', age=30)
user.save()

# Create a new post
post = Post(title='My First Post', author=user)
post.save()

# Create a new comment and add it to the post
comment = Comment(content='Nice post!', author=user)
post.comments.append(comment)
post.save()


Now, let's query for the post and print out the comments:

In [ ]:
# Get the first post
post = Post.objects.first()

# Print post details
print(f'Post: {post.title}, Author: {post.author.name}, Posted at: {post.posted_at}')

# Print all comments
for comment in post.comments:
    print(f'Comment: {comment.content}, Author: {comment.author.name}, Posted at: {comment.posted_at}')
